In [1]:
# pip install tensorflow

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import cv2

import warnings
warnings.filterwarnings("ignore")

from PIL import Image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
from tensorflow.keras.utils import load_img
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Dropout, concatenate, Conv2DTranspose
import random
import glob

from sklearn.model_selection import train_test_split

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
print(tf.__version__)

2.11.0


In [4]:
masks = glob.glob("/kaggle/input/breast-ultrasound-images-dataset/Dataset_BUSI_with_GT/*/*_mask.png")

In [5]:
masks[:4]

['/kaggle/input/breast-ultrasound-images-dataset/Dataset_BUSI_with_GT/benign/benign (166)_mask.png',
 '/kaggle/input/breast-ultrasound-images-dataset/Dataset_BUSI_with_GT/benign/benign (290)_mask.png',
 '/kaggle/input/breast-ultrasound-images-dataset/Dataset_BUSI_with_GT/benign/benign (430)_mask.png',
 '/kaggle/input/breast-ultrasound-images-dataset/Dataset_BUSI_with_GT/benign/benign (89)_mask.png']

In [6]:
images = [mask_images.replace("_mask", "") for mask_images in masks]

In [7]:
images[:4]

['/kaggle/input/breast-ultrasound-images-dataset/Dataset_BUSI_with_GT/benign/benign (166).png',
 '/kaggle/input/breast-ultrasound-images-dataset/Dataset_BUSI_with_GT/benign/benign (290).png',
 '/kaggle/input/breast-ultrasound-images-dataset/Dataset_BUSI_with_GT/benign/benign (430).png',
 '/kaggle/input/breast-ultrasound-images-dataset/Dataset_BUSI_with_GT/benign/benign (89).png']

In [8]:
series = list(zip(images, masks))

In [9]:
series[:4]

[('/kaggle/input/breast-ultrasound-images-dataset/Dataset_BUSI_with_GT/benign/benign (166).png',
  '/kaggle/input/breast-ultrasound-images-dataset/Dataset_BUSI_with_GT/benign/benign (166)_mask.png'),
 ('/kaggle/input/breast-ultrasound-images-dataset/Dataset_BUSI_with_GT/benign/benign (290).png',
  '/kaggle/input/breast-ultrasound-images-dataset/Dataset_BUSI_with_GT/benign/benign (290)_mask.png'),
 ('/kaggle/input/breast-ultrasound-images-dataset/Dataset_BUSI_with_GT/benign/benign (430).png',
  '/kaggle/input/breast-ultrasound-images-dataset/Dataset_BUSI_with_GT/benign/benign (430)_mask.png'),
 ('/kaggle/input/breast-ultrasound-images-dataset/Dataset_BUSI_with_GT/benign/benign (89).png',
  '/kaggle/input/breast-ultrasound-images-dataset/Dataset_BUSI_with_GT/benign/benign (89)_mask.png')]

In [10]:
dataset = pd.DataFrame(series, columns=['images', 'masks'])

In [11]:
dataset

,images,masks
0,/kaggle/input/breast-ultrasound-images-dataset...,/kaggle/input/breast-ultrasound-images-dataset...
1,/kaggle/input/breast-ultrasound-images-dataset...,/kaggle/input/breast-ultrasound-images-dataset...
2,/kaggle/input/breast-ultrasound-images-dataset...,/kaggle/input/breast-ultrasound-images-dataset...
3,/kaggle/input/breast-ultrasound-images-dataset...,/kaggle/input/breast-ultrasound-images-dataset...
4,/kaggle/input/breast-ultrasound-images-dataset...,/kaggle/input/breast-ultrasound-images-dataset...
...,...,...
775,/kaggle/input/breast-ultrasound-images-dataset...,/kaggle/input/breast-ultrasound-images-dataset...
776,/kaggle/input/breast-ultrasound-images-dataset...,/kaggle/input/breast-ultrasound-images-dataset...
777,/kaggle/input/breast-ultrasound-images-dataset...,/kaggle/input/breast-ultrasound-images-dataset...
778,/kaggle/input/breast-ultrasound-images-dataset...,/kaggle/input/breast-ultrasound-images-dataset...


In [12]:
train, val = train_test_split(dataset, test_size=0.25)

In [13]:
if any(data in val['images'] for data in train['images']):
    print('There\'s a problem in splitting, possible data leakage')
else:
    print('The data has been splited correctly')

The data has been splited correctly


In [14]:
# from tensorflow.keras.utils import image_dataset_from_directory

# train_data = image_dataset_from_directory(
#                   train,
#                   validation_split=0.2,
#                   subset="training",
#                   seed=123,
#                   image_size=(224, 224),
#                   batch_size=32)

In [15]:
# batch_size = 32

# train_data = tf.data.Dataset.from_tensor_slices(train)
# train_data = train_data.map(lambda x: load_and_preprocess_image(x[0], x[1]))
# train_data = train_data.shuffle(buffer_size=10000)
# train_data = train_data.batch(batch_size)
# train_data = train_data.prefetch(buffer_size=tf.data.AUTOTUNE)

# val_data = tf.data.Dataset.from_tensor_slices(val)
# val_data = val_data.map(lambda x: load_and_preprocess_image(x[0], x[1]))
# val_data = val_data.shuffle(buffer_size=10000)
# val_data = val_data.batch(batch_size)
# val_data = val_data.prefetch(buffer_size=tf.data.AUTOTUNE)

In [16]:
def load_and_preprocess_image(image_path, mask_path):
    # Load and preprocess the image
    image = tf.io.read_file(image_path)
    image = tf.image.decode_png(image, channels=3)  # Adjust channels as needed
    image = tf.image.convert_image_dtype(image, tf.float32)
    image = tf.image.resize(image, (256, 256))  # Adjust the size as needed

    # Load and preprocess the mask
    mask = tf.io.read_file(mask_path)
    mask = tf.image.decode_png(mask, channels=1)  # Masks are typically grayscale
    mask = tf.image.convert_image_dtype(mask, tf.float32)
    mask = tf.image.resize(mask, (256, 256))  # Adjust the size as needed

    return image, mask

In [44]:
train = tf.data.Dataset.from_tensor_slices((train['images'], train['masks']))
train = train.map(load_and_preprocess_image)
train_data = train.shuffle(buffer_size=10000)
train_data = train_data.batch(batch_size=32)
train_data = train_data.prefetch(buffer_size=tf.data.AUTOTUNE)

In [45]:
val = tf.data.Dataset.from_tensor_slices((val['images'], val['masks']))
val = val.map(load_and_preprocess_image)
val_data = val.shuffle(buffer_size=10000)
val_data = val_data.batch(batch_size=32)
val_data = val_data.prefetch(buffer_size=tf.data.AUTOTUNE)

In [46]:
class UNet:
    def __init__(self, input_shape, num_classes):
        self.input_shape = input_shape
        self.num_classes = num_classes
        self.model = self.build_model()

    def conv_block(self, inputs, filters, kernel_size=3, activation='relu', padding='same'):
        conv = Conv2D(filters, kernel_size, activation=activation, padding=padding)(inputs)
        conv = Conv2D(filters, kernel_size, activation=activation, padding=padding)(conv)
        return conv

    def build_model(self):
        inputs = Input(self.input_shape)

        # Encoder
        conv1 = self.conv_block(inputs, 64)
        pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

        conv2 = self.conv_block(pool1, 128)
        pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

        conv3 = self.conv_block(pool2, 256)
        pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

        conv4 = self.conv_block(pool3, 512)
        pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

        conv5 = self.conv_block(pool4, 1024)

        # Decoder
        up6 = Conv2DTranspose(512, (2, 2), strides=(2, 2), padding='same')(conv5)
        concat6 = concatenate([conv4, up6], axis=3)
        conv6 = self.conv_block(concat6, 512)

        up7 = Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(conv6)
        concat7 = concatenate([conv3, up7], axis=3)
        conv7 = self.conv_block(concat7, 256)

        up8 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(conv7)
        concat8 = concatenate([conv2, up8], axis=3)
        conv8 = self.conv_block(concat8, 128)

        up9 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(conv8)
        concat9 = concatenate([conv1, up9], axis=3)
        conv9 = self.conv_block(concat9, 64)

        # Output
        outputs = Conv2D(self.num_classes, (1, 1), activation='softmax')(conv9)

        model = Model(inputs=inputs, outputs=outputs)
        return model

In [50]:
input_shape = (256, 256, 3)
num_classes = 1
model = UNet(input_shape, num_classes).model

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.fit(train_data, epochs=10, validation_data=val_data)

Epoch 1/10
19/19 [==============================] - 59s 1s/step - loss: 0.6891 - accuracy: 0.0771 - val_loss: 0.3217 - val_accuracy: 0.0730
Epoch 2/10
19/19 [==============================] - 20s 802ms/step - loss: 0.3078 - accuracy: 0.0771 - val_loss: 0.2493 - val_accuracy: 0.0730
Epoch 3/10
19/19 [==============================] - 20s 801ms/step - loss: 0.2717 - accuracy: 0.0771 - val_loss: 0.2372 - val_accuracy: 0.0730
Epoch 4/10
19/19 [==============================] - 20s 797ms/step - loss: 0.2438 - accuracy: 0.0771 - val_loss: 0.2334 - val_accuracy: 0.0730
Epoch 5/10
19/19 [==============================] - 20s 802ms/step - loss: 0.2313 - accuracy: 0.0771 - val_loss: 0.2152 - val_accuracy: 0.0730
Epoch 6/10


KeyboardInterrupt: 